# Testing norfair re-id algorithms

In [ ]:
# !pip install norfair

In [5]:
!pip install detectron2

ERROR: Could not find a version that satisfies the requirement detectron2 (from versions: none)
ERROR: No matching distribution found for detectron2


In [ ]:
import cv2
import numpy as np

In [4]:
from detectron2.config import get_cfg
from detectron2.engine import DefaultPredictor

ModuleNotFoundError: No module named 'detectron2'

In [6]:
Detection?

Init signature:
Detection(
    points: numpy.ndarray,
    scores: numpy.ndarray = None,
    data: Any = None,
    label: Hashable = None,
    embedding=None,
)
Docstring:     
Detections returned by the detector must be converted to a `Detection` object before being used by Norfair.

Parameters
----------
points : np.ndarray
    Points detected. Must be a rank 2 array with shape `(n_points, n_dimensions)` where n_dimensions is 2 or 3.
scores : np.ndarray, optional
    An array of length `n_points` which assigns a score to each of the points defined in `points`.

    This is used to inform the tracker of which points to ignore;
    any point with a score below `detection_threshold` will be ignored.

    This useful for cases in which detections don't always have every point present, as is often the case in pose estimators.
data : Any, optional
    The place to store any extra data which may be useful when calculating the distance function.
    Anything stored here will be available to u

In [ ]:
from norfair import Detection, Tracker, Video, draw_tracked_objects

# Set up Detectron2 object detector
cfg = get_cfg()
cfg.merge_from_file("demos/faster_rcnn_R_50_FPN_3x.yaml")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5
cfg.MODEL.WEIGHTS = "detectron2://COCO-InstanceSegmentation/mask_rcnn_R_50_FPN_3x/137849600/model_final_f10217.pkl"
detector = DefaultPredictor(cfg)

# Norfair
video = Video(input_path="video.mp4")
tracker = Tracker(distance_function="euclidean", distance_threshold=20)

for frame in video:
    detections = detector(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    detections = [Detection(p) for p in detections['instances'].pred_boxes.get_centers().cpu().numpy()]
    tracked_objects = tracker.update(detections=detections)
    draw_tracked_objects(frame, tracked_objects)
    video.write(frame)